In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
def distanceFunc(X, MU):
    # Inputs
    # X: is an NxD matrix (N observations and D dimensions)
    # MU: is an KxD matrix (K means and D dimensions)
    # Outputs
    # pair_dist: is the squared pairwise distance matrix (NxK)
    # TODO
    pair_dist = tf.transpose(tf.reduce_sum(tf.square(X - MU), axis=2))
    return pair_dist

In [ ]:
def log_GaussPDF(X, mu, sigma):
    # Inputs
    # X: N X D
    # mu: K X D
    # sigma: K X 1

    # Outputs:
    # log Gaussian PDF N X K

    # TODO
    dim = data.shape[1]
    dist = distanceFunc(X, mu)
    sigma = tf.transpose(sigma)
    gauss_coeff = tf.log(2 * np.pi * sigma)
    pdf = -(1/2)*dim*gauss_coeff - (dist/(2*sigma))
    return pdf

In [ ]:
def log_posterior(log_PDF, log_pi):
    # Input
    # log_PDF: log Gaussian PDF N X K
    # log_pi: K X 1

    # Outputs
    # log_post: N X K

    # TODO
    log_pi = tf.transpose(log_pi)
    prob = log_pi + log_PDF
    prob_sum = hlp.reduce_logsumexp(prob, keep_dims=True)
    posterior = prob - prob_sum
    return posterior

In [ ]:
def loss_function(log_PDF, log_pi):
    loss = -tf.reduce_sum(hlp.reduce_logsumexp(log_PDF + log_pi, 1, keep_dims=True), axis =0)
    return loss